# Partie Thermique

---

In [44]:
from numpy import linspace, zeros
from numpy import linalg
from numpy import ndarray
import plotly.express as px

In [45]:
#Initialisation des constante

lbda = 0.96
rho = 2550
Cp = 840

N = 100
R = 1

t = 10
Nt = 10000

Text = linspace(1770,1670, Nt)
Tref = 1770

In [47]:

for m in range(1, Nt):

    T[m][0] = W * dt * (T[m-1][1] - T[m-1][0])/sq(dr) + T[m-1][0]

    for i in range(1,N):
        T[m][i] = W * dt * ( (T[m-1][i+1] - 2*T[m-1][i] + T[m-1][i-1])/sq(dr) + (4*(T[m-1][i+1] - T[m-1][i])) / ( sq(dr) * (2*i - 1))) + T[m-1][i]

In [48]:
#Tim = [ [ (T[m][i] + T[m][i-1])/2 for i in range(1, N+0)] for m in range(0, Nt)]
totalDT = [ [Tref - T[m][i] for i in range(len(T[0])-1)] for m in range(len(T))]

In [49]:
X = linspace(0, R, N)
Y = T[-1][:-1]

px.line(x = X , y = Y)

In [50]:
totalDT = []
X = linspace(0, R, N)
totalDT.append([ -(r*r/(R*R) * 100) for r in X] )
#totalDT.append([0 for i in X])

#px.line(x = X, y = DT, title=r"Temperature en focntion de R")

# Partie Mécanique

---

In [60]:
p = 0

E = 71e9
nu = 0.21
#kc = E/(3*(1-2*nu))
#muc= E/(2*(1+nu))

kc = 71e9/(3*(1-2*0.21))
muc = 71e9/(2*(1+0.21))

alpha_c=9e-6

In [61]:
def k(i) -> float:
    return kc

def mu(i) -> float:
    return muc


def A(i) -> float:
    A = 4/(9*k(i)) + 1/(12 * mu(i))
    return A

def B(i) -> float:
    B = 1/(3*k(i))
    return B

def alpha(i) -> float:
    return alpha_c


## Construction de M et de R

In [62]:

temps = 0#Nt-1
DT = totalDT[temps]

M = [[0 for i in range(2*N)] for j in range(2*N)]
Rmat = [0]*(2*N)

M[0][1] = 1

M[2*N-1][2*N-2] = "-3*k(N)"
M[2*N-1][2*N-1] = "-4*mu(4)/((dr*N)**3)"
Rmat[2*N-1] = "p - 3*k(N)*alpha(N)*DT[N-1]"

M[2*N-1][2*N-2] = -3*k(N)
M[2*N-1][2*N-1] = 4*mu(4)/((dr*N)**3)
Rmat[2*N-1] = p - 3*k(N)*alpha(N)*DT[N-1]



for i in range(1, N):
    r = dr * i
    M[ 2*(i-1) + 1][ 2*(i-1) : 2*(i-1) + 4] = [-r, -1/sq(r), r, 1/sq(r)]
    #M[ 2*(i-1) + 1][ 2*(i-1) : 2*(i-1) + 4] = ["-r"+str(i), "-1/r"+str(i)+"²","r"+str(i), "1/r"+str(i)+"²"]

for i in range(1,N):
    r = dr*i
    M[2*(i-1)+2][2*(i-1) : 2*(i-1) + 4] = [-3*k(i), 4*mu(i)/(r**3), 3*k(i+1), -4*mu(i+1)/(r**3)]
    #M[2*(i-1)+2][2*(i-1) : 2*(i-1) + 4] = ["-3k"+str(i), "4mu"+str(i)+"/(r**3)", "3k"+str(i+1), "-4mu"+str(i+1)+"/(r**3)"]

    Rmat[2*(i-1)+2] = -3*k(i)*alpha(i)*DT[i-1]+3*k(i+1)*alpha(i+1)*DT[i]
    #Rmat[2*(i-1)+2] = "-3k"+str(i)+"*alpha"+str(i)+"*DT"+str(i)+"+3k"+str(i+1)+"*alpha"+str(i+1)+"*DT"+str(i)

In [65]:
def Tcontraintesi(sol_ab,i,r) -> ndarray:
    '''Calcule de la contrainte a partir des solution
    Attention i>0'''

    sigmai=zeros((3,3))
    ai,bi=sol_ab[2*i-2],sol_ab[2*i-1]
    sigmai[0][0]= -4*mu(i)*bi/(r**3) + 3*k(i)*ai - 3*k(i)*alpha(i)*DT[i-1]
    sigmai[1][1]=  2*mu(i)*bi/(r**3) + 3*k(i)*ai - 3*k(i)*alpha(i)*DT[i-1]
    sigmai[2][2]=  2*mu(i)*bi/(r**3) + 3*k(i)*ai - 3*k(i)*alpha(i)*DT[i-1]

    return sigmai



def Tdeplacementsi(sol_ab,i,r) -> ndarray:
    '''Calcule de la contrainte a partir des solution
    Attention i>0'''

    epsilonai=zeros((3,3))
    ai,bi=sol_ab[2*i-2],sol_ab[2*i-1]
    epsilonai[0][0]=ai - 2 * bi / (r**3)
    epsilonai[1][1]=ai + bi/(r**3)
    epsilonai[2][2]=ai + bi/(r**3)

    return epsilonai


def deplacement(sol_ab, i, r):
    ai,bi=sol_ab[2*i-2],sol_ab[2*i-1]
    return ai * r + bi/(r**2)

In [66]:
sol = linalg.solve(M, Rmat)

X = linspace(0, R, N)
Y1, Y2, Y3, Y4 = [], [], [], []

for i in range(1, N+1):
    r = i*dr
    contrainte = Tcontraintesi(sol, i, r)
    Y1.append(contrainte[0][0])
    Y2.append(contrainte[1][1])
    Y3.append(contrainte[2][2])
    
    Y4.append(deplacement(sol, i, r))

px.line(x = X, y = [Y1, Y2, Y3])
#px.line(x = X, y = Y4)